# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [1]:
# https://jportal.mdcourts.gov/license/pbPublicSearch.jsp

**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages.

### How will you identify the checkbox to check it?

Selenium can find elements by:

- name
- Class
- ID
- CSS selector (**ASK ME WHAT THIS IS** if you don't know)
- XPath (**ASK ME WHAT THIS IS** because you definitely don't know)
- Link text
- Partial link text

So in other words, what's unique about this element?

- *TIP: I have a secret awesome way to do this, but you have to ask.*

In [2]:
# driver.find_element_by_xpath('//*[@id="checkbox"]').click()

### How will you identify the button to select it, or the form to submit it?

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

In [3]:
# driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]').click()

### Once you're on the next page, how will you click the "SEARCH LICENSE RECORDS" link or follow its URL?

In [4]:
# driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[6]/td[2]/a[2]').click()

### Now you're on the form page. How do you pick "Statewide" for the jurisdiction dropdown?

In [5]:
# Select(driver.find_element_by_xpath('//*[@id="slcJurisdiction"]')).select.select_by_visible_taxt('Statewide)

### How do you type "vap%" into the Trade Name field?

In [6]:
# driver.find_element_by_xpath('//*[@id="txtTradeName"]').send_keys('vap%')

### How do you click the submit button or submit the form?

In [7]:
# driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]').click()

### How can you find and click the 'Next' button on the search results page?

In [8]:
# driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr/td[3]/a/nobr').click()

# Okay, let's scrape!

### Use Selenium to search for vape shops statewide, and then click through until it reaches the last page.

Don't scrape yet!

In [9]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd

In [10]:
driver = webdriver.Chrome()

In [11]:
driver.get("https://jportal.mdcourts.gov/license/pbPublicSearch.jsp")

In [12]:
driver.find_element_by_xpath('//*[@id="checkbox"]').click()

In [13]:
driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]').click()

In [14]:
driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[6]/td[2]/a[2]').click()

In [15]:
Select(driver.find_element_by_xpath('//*[@id="slcJurisdiction"]')).select_by_visible_text('Statewide')

In [16]:
driver.find_element_by_xpath('//*[@id="txtTradeName"]').send_keys('vap%')

In [18]:
driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]').click()

In [19]:
while True:
    try:
        driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr/td[3]/a').click()
    except:
        break

### Use Selenium to scrape the first page of search results for vape shops statewide (well, we'll try).

- *TIP: You can find elements/text using Selenium, or use BeautifulSoup with `doc = BeautifulSoup(driver.page_source)`*

I've included some BeautifulSoup code that might be helpful. If you use it, **ask me how I made it.** It's important.

In [20]:
# first go back to the first page
while True:
    try:
        driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr/td[1]/a').click()
    except:
        break

In [21]:
doc = BeautifulSoup(driver.page_source, 'html.parser')

licenses_first_page = []

for item in doc.find_all(class_='searchfieldtitle'):
    license = {}
    if item.find_all('td')[1].text:
        license['company'] = item.find_all('td')[1].text
    if item.a:
        license['url'] = item.a['href']
    if item.find_next_siblings('tr')[0].find_all('td')[1].text:
        license['name'] = item.find_next_siblings('tr')[0].find_all('td')[1].text
    if item.find_next_siblings('tr')[1].find_all('td')[1].text:
        license['address_street'] = item.find_next_siblings('tr')[1].find_all('td')[1].text
    if item.find_next_siblings('tr')[2].find_all('td')[1].text:
        license['address_city'] = item.find_next_siblings('tr')[2].find_all('td')[1].text
    if item.find_next_siblings('tr')[3].find_all('td')[1].text:
        license['address_county'] = item.find_next_siblings('tr')[3].find_all('td')[1].text
    if item.find_next_siblings('tr')[0].find_all('td')[2].text:
        license['status'] = item.find_next_siblings('tr')[0].find_all('td')[2].text[13:]
    if item.find_next_siblings('tr')[1].find_all('td')[2].text:
        license['id'] = item.find_next_siblings('tr')[1].find_all('td')[2].text[9:]
    if item.find_next_siblings('tr')[2].find_all('td')[2].text:
        license['date'] = item.find_next_siblings('tr')[2].find_all('td')[2].text[13:]
    licenses_first_page.append(license)

licenses_first_page

[{'address_city': 'SALISBURY, MD 21801',
  'address_county': 'Wicomico County',
  'address_street': '1724 N SALISBURY BLVD UNIT 2',
  'company': 'VAPE IT STORE I',
  'date': '4/27/2017',
  'id': '22173807',
  'name': 'AMIN NARGIS',
  'status': 'Issued',
  'url': 'pbLicenseDetail.jsp?owi=Kch8QLy44P8%3D'},
 {'address_city': 'SALISBURY, MD 21801',
  'address_county': 'Wicomico County',
  'address_street': '1015 S SALISBURY BLVD',
  'company': 'VAPE IT STORE II',
  'date': '4/27/2017',
  'id': '22173808',
  'name': 'AMIN NARGIS',
  'status': 'Issued',
  'url': 'pbLicenseDetail.jsp?owi=uuOm%2B41TYQE%3D'},
 {'address_city': 'GAMBRILLS, MD 21054',
  'address_county': 'Anne Arundel County',
  'address_street': '2299 JOHNS HOPKINS ROAD',
  'company': 'VAPEPAD THE',
  'date': '4/05/2017',
  'id': '02104436',
  'name': 'ANJ DISTRIBUTIONS LLC',
  'status': 'Issued',
  'url': 'pbLicenseDetail.jsp?owi=9ht3HmZUqTo%3D'},
 {'address_city': 'CHESTER, MD 21619',
  'address_county': "Queen Anne's County",

### Save these into `vape-results.csv`

In [22]:
df = pd.DataFrame(licenses_first_page)
df.to_csv('vape-results.csv', index=False)

### Open `vape-results.csv` to make sure there aren't any extra weird columns

In [23]:
pd.read_csv('vape-results.csv')

,address_city,address_county,address_street,company,date,id,name,status,url
0,"SALISBURY, MD 21801",Wicomico County,1724 N SALISBURY BLVD UNIT 2,VAPE IT STORE I,4/27/2017,22173807.0,AMIN NARGIS,Issued,pbLicenseDetail.jsp?owi=Kch8QLy44P8%3D
1,"SALISBURY, MD 21801",Wicomico County,1015 S SALISBURY BLVD,VAPE IT STORE II,4/27/2017,22173808.0,AMIN NARGIS,Issued,pbLicenseDetail.jsp?owi=uuOm%2B41TYQE%3D
2,"GAMBRILLS, MD 21054",Anne Arundel County,2299 JOHNS HOPKINS ROAD,VAPEPAD THE,4/05/2017,2104436.0,ANJ DISTRIBUTIONS LLC,Issued,pbLicenseDetail.jsp?owi=9ht3HmZUqTo%3D
3,"CHESTER, MD 21619",Queen Anne's County,110 S. PINEY RD,VAPE FROG,5/31/2017,17165957.0,COX TRADING COMPANY L L C,Issued,pbLicenseDetail.jsp?owi=7JaLnZGqcN4%3D
4,"SEVERNA PARK, MD 21146",Anne Arundel County,346 RITCHIE HIGHWAY,VAPE FROG,NaN,NaN,COX TRADING LLC,Pending,NaN


## Use Selenium to scrape ALL pages of results, save the results into `vape-results-all.csv`.

In [24]:
licenses = []

while True:
    doc = BeautifulSoup(driver.page_source, 'html.parser')
    for item in doc.find_all(class_='searchfieldtitle'):
        license = {}
        if item.find_all('td')[1].text:
            license['company'] = item.find_all('td')[1].text
        if item.a:
            license['url'] = item.a['href']
        if item.find_next_siblings('tr')[0].find_all('td')[1].text:
            license['name'] = item.find_next_siblings('tr')[0].find_all('td')[1].text
        if item.find_next_siblings('tr')[1].find_all('td')[1].text:
            license['address_street'] = item.find_next_siblings('tr')[1].find_all('td')[1].text
        if item.find_next_siblings('tr')[2].find_all('td')[1].text:
            license['address_city'] = item.find_next_siblings('tr')[2].find_all('td')[1].text
        if item.find_next_siblings('tr')[3].find_all('td')[1].text:
            license['address_county'] = item.find_next_siblings('tr')[3].find_all('td')[1].text
        if item.find_next_siblings('tr')[0].find_all('td')[2].text:
            license['status'] = item.find_next_siblings('tr')[0].find_all('td')[2].text[13:]
        if item.find_next_siblings('tr')[1].find_all('td')[2].text:
            license['id'] = item.find_next_siblings('tr')[1].find_all('td')[2].text[9:]
        if item.find_next_siblings('tr')[2].find_all('td')[2].text:
            license['date'] = item.find_next_siblings('tr')[2].find_all('td')[2].text[13:]
        licenses.append(license)
    try:
        driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr/td[3]/a').click()
    except:
        break

df = pd.DataFrame(licenses)
df.to_csv('vape-results-all.csv', index=False)
pd.read_csv('vape-results-all.csv')

,address_city,address_county,address_street,company,date,id,name,status,url
0,"SALISBURY, MD 21801",Wicomico County,1724 N SALISBURY BLVD UNIT 2,VAPE IT STORE I,4/27/2017,22173807.0,AMIN NARGIS,Issued,pbLicenseDetail.jsp?owi=Kch8QLy44P8%3D
1,"SALISBURY, MD 21801",Wicomico County,1015 S SALISBURY BLVD,VAPE IT STORE II,4/27/2017,22173808.0,AMIN NARGIS,Issued,pbLicenseDetail.jsp?owi=uuOm%2B41TYQE%3D
2,"GAMBRILLS, MD 21054",Anne Arundel County,2299 JOHNS HOPKINS ROAD,VAPEPAD THE,4/05/2017,2104436.0,ANJ DISTRIBUTIONS LLC,Issued,pbLicenseDetail.jsp?owi=9ht3HmZUqTo%3D
3,"CHESTER, MD 21619",Queen Anne's County,110 S. PINEY RD,VAPE FROG,5/31/2017,17165957.0,COX TRADING COMPANY L L C,Issued,pbLicenseDetail.jsp?owi=7JaLnZGqcN4%3D
4,"SEVERNA PARK, MD 21146",Anne Arundel County,346 RITCHIE HIGHWAY,VAPE FROG,NaN,NaN,COX TRADING LLC,Pending,NaN
5,"SALISBURY, MD 21801",Wicomico County,1724 N SALISBURY BLVD UNIT 2,VAPE IT STORE I,4/27/2017,22173807.0,AMIN NARGIS,Issued,pbLicenseDetail.jsp?owi=Kch8QLy44P8%3D
6,"SALISBURY, MD 21801",Wicomico County,1015 S SALISBURY BLVD,VAPE IT STORE II,4/27/2017,22173808.0,AMIN NARGIS,Issued,pbLicenseDetail.jsp?owi=uuOm%2B41TYQE%3D
7,"GAMBRILLS, MD 21054",Anne Arundel County,2299 JOHNS HOPKINS ROAD,VAPEPAD THE,4/05/2017,2104436.0,ANJ DISTRIBUTIONS LLC,Issued,pbLicenseDetail.jsp?owi=9ht3HmZUqTo%3D
8,"CHESTER, MD 21619",Queen Anne's County,110 S. PINEY RD,VAPE FROG,5/31/2017,17165957.0,COX TRADING COMPANY L L C,Issued,pbLicenseDetail.jsp?owi=7JaLnZGqcN4%3D
9,"SEVERNA PARK, MD 21146",Anne Arundel County,346 RITCHIE HIGHWAY,VAPE FROG,NaN,NaN,COX TRADING LLC,Pending,NaN


In [25]:
driver.close()